# Testing new parameters on 1,000 maps through TI9

Testing different logistic scale and time scale (for covariance function) parameter values using AUC and likelihood.

## Inputs

### Input files

In [1]:
MATCHES_DF = 'data/raw/premium_matches.2019-08-31.json.gz'

In [23]:
from collections import OrderedDict
files = OrderedDict()
for ls in [0.5, 1, 2, 3, 4.5, 6, 8]:
    files['ls = ' + str(ls)] = (
        ls,  # Logistic scale
        f'fitted/ti9_full_new_params_by_series.ls-{ls}.win_probs.gz',
        f'fitted/ti9_full_new_params_by_series.ls-{ls}.player_skills'
            '.gz'
    )
for ls in [2, 2.333, 2.667, 3, 3.333, 3.667, 4, 4.333, 4.667]:
    files['ls = ' + str(ls) + ' (new)'] = (
        ls,  # Logistic scale
        f'fitted/ti9_1000_maps_by_series.ls-{ls}.win_probs.gz',
        f'fitted/ti9_1000_maps_by_series.ls-{ls}.player_skills'
            '.gz'
    )
for ts in [0.5, 0.75, 1, 1.25, 1.5, 2, 3]:
    files['ts = ' + str(ts)] = (
        3.0,  # Logistic scale
        f'fitted/ti9_full_new_params_by_series.ts-{ts}.win_probs.gz',
        f'fitted/ti9_full_new_params_by_series.ts-{ts}.player_skills.gz'
    )
for ts in [1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25]:
    files['ts = ' + str(ts) + ' (new)'] = (
        3.0,  # Logistic scale
        f'fitted/ti9_1000_maps_by_series.ts-{ts}.win_probs.gz',
        f'fitted/ti9_1000_maps_by_series.ts-{ts}.player_skills.gz'
    )
files

OrderedDict([('ls = 0.5',
              (0.5,
               'fitted/ti9_full_new_params_by_series.ls-0.5.win_probs.gz',
               'fitted/ti9_full_new_params_by_series.ls-0.5.player_skills.gz')),
             ('ls = 1',
              (1,
               'fitted/ti9_full_new_params_by_series.ls-1.win_probs.gz',
               'fitted/ti9_full_new_params_by_series.ls-1.player_skills.gz')),
             ('ls = 2',
              (2,
               'fitted/ti9_full_new_params_by_series.ls-2.win_probs.gz',
               'fitted/ti9_full_new_params_by_series.ls-2.player_skills.gz')),
             ('ls = 3',
              (3,
               'fitted/ti9_full_new_params_by_series.ls-3.win_probs.gz',
               'fitted/ti9_full_new_params_by_series.ls-3.player_skills.gz')),
             ('ls = 4.5',
              (4.5,
               'fitted/ti9_full_new_params_by_series.ls-4.5.win_probs.gz',
               'fitted/ti9_full_new_params_by_series.ls-4.5.player_skills.gz')),
             (

### Parameters

In [3]:
# Scaling factor for logistic regression win probability.
# LOGISTIC_SCALE = 3.0

# Maximum difference in minutes between the Fairlay closing datetime and the
# Datdota map start datetime.
MAX_TIMEDIFF_MINS = 1440

## Setup.

### Load libraries.

In [4]:
import os
while 'src' not in os.listdir():
    os.chdir("..")

In [5]:
import gzip
import json
import pickle

import numpy as np
import pandas as pd
import scipy

import src.fairlay
import src.load
import src.statics
import src.stats

### Load data.

#### Matches and predictions

In [24]:
with gzip.open(MATCHES_DF, 'rb') as fh:
    matches = src.load.MatchDF.from_json(json.load(fh)['data'])

In [25]:
match_pred_objs = OrderedDict()
for label, val in files.items():
    logistic_scale, pred_file, skills_file = val
    pred_df = pd.read_csv(pred_file, header=0, sep='\t', index_col=0)
    skills_mat = pd.read_csv(skills_file, header=0, sep='\t', index_col=0)
    skills_mat.columns = skills_mat.columns.astype(int)
    match_pred_objs[label] = \
        src.stats.MatchPred(matches, pred_df, logistic_scale, skills_mat)

## AUCs

In [26]:
aucs = []
for label, match_pred in match_pred_objs.items():
    auc = match_pred._roc_auc(
        False,
        match_pred.df.iter > 0)
    aucs.append(auc)

#     print("--------")
#     print(label)
#     print("TI9 group stage AUC: {:.3f}".format(ti9_group_stage_auc))
#     print("TI9 playoffs AUC: {:.3f}".format(ti9_playoffs_auc))

pd.Series(aucs, index=match_pred_objs.keys())

ls = 0.5            0.645280
ls = 1              0.658237
ls = 2              0.667028
ls = 3              0.669351
ls = 4.5            0.667513
ls = 6              0.663592
ls = 8              0.658513
ls = 2 (new)        0.666624
ls = 2.333 (new)    0.668318
ls = 2.667 (new)    0.669051
ls = 3 (new)        0.669452
ls = 3.333 (new)    0.669624
ls = 3.667 (new)    0.669043
ls = 4 (new)        0.668070
ls = 4.333 (new)    0.668002
ls = 4.667 (new)    0.667109
ts = 0.5            0.663824
ts = 0.75           0.666740
ts = 1              0.668550
ts = 1.25           0.669351
ts = 1.5            0.670185
ts = 2              0.670513
ts = 3              0.669688
ts = 1.25 (new)     0.669452
ts = 1.5 (new)      0.670405
ts = 1.75 (new)     0.670497
ts = 2 (new)        0.670769
ts = 2.25 (new)     0.670549
ts = 2.5 (new)      0.669932
ts = 2.75 (new)     0.669988
ts = 3 (new)        0.669728
ts = 3.25 (new)     0.669720
dtype: float64

## Likelihoods

In [27]:
logliks = []
for label, match_pred in match_pred_objs.items():
    loglik = sum(match_pred.match_loglik(False)[match_pred.df.iter > 0])
    logliks.append(loglik)
pd.Series(logliks, index=match_pred_objs.keys())

ls = 0.5           -743.879784
ls = 1             -681.845221
ls = 2             -653.947995
ls = 3             -649.853927
ls = 4.5           -651.762285
ls = 6             -656.031617
ls = 8             -662.020677
ls = 2 (new)       -654.219769
ls = 2.333 (new)   -651.385326
ls = 2.667 (new)   -650.085988
ls = 3 (new)       -649.813906
ls = 3.333 (new)   -649.713103
ls = 3.667 (new)   -649.940528
ls = 4 (new)       -650.648446
ls = 4.333 (new)   -651.315518
ls = 4.667 (new)   -652.217778
ts = 0.5           -652.936902
ts = 0.75          -651.012802
ts = 1             -650.182926
ts = 1.25          -649.853927
ts = 1.5           -649.547137
ts = 2             -649.294076
ts = 3             -649.336656
ts = 1.25 (new)    -649.813906
ts = 1.5 (new)     -649.431415
ts = 1.75 (new)    -649.479222
ts = 2 (new)       -649.115407
ts = 2.25 (new)    -649.257168
ts = 2.5 (new)     -649.273786
ts = 2.75 (new)    -649.283096
ts = 3 (new)       -649.283710
ts = 3.25 (new)    -649.376761
dtype: f

## Bias tables

### Varying logistic scale

In [28]:
for label, match_pred in match_pred_objs.items():
    if not label.startswith('ls'):
        continue
    print(label)
    cur_loc = match_pred.df.iter > 0
    display(match_pred.match_pred_df_bias(loc=cur_loc,
                                          assume_side_known=True))

ls = 0.5


,n,obs,exp,poibin_pval,pois_pval
"(-0.001, 0.1]",71.0,22.0,3.831,1.209255e-12,5.191492e-11
"(0.1, 0.2]",68.0,24.0,9.946,1.003116e-05,8.633888e-05
"(0.2, 0.3]",80.0,30.0,19.586,6.393453e-03,2.072855e-02
"(0.3, 0.4]",95.0,37.0,33.310,3.656380e-01,4.593960e-01
"(0.4, 0.5]",136.0,70.0,62.336,1.597572e-01,3.014518e-01
"(0.5, 0.6]",149.0,81.0,81.873,9.488010e-01,9.817967e-01
"(0.6, 0.7]",124.0,66.0,80.423,9.854754e-03,1.137026e-01
"(0.7, 0.8]",99.0,64.0,74.418,2.481747e-02,2.473113e-01
"(0.8, 0.9]",90.0,59.0,76.321,9.145790e-06,4.729596e-02
"(0.9, 1.0]",88.0,65.0,83.671,2.522214e-11,4.067581e-02


ls = 1


,n,obs,exp,poibin_pval,pois_pval
"(-0.001, 0.1]",23.0,9.0,1.545,0.000002,0.000011
"(0.1, 0.2]",63.0,15.0,9.510,0.044650,0.067454
"(0.2, 0.3]",86.0,29.0,21.885,0.064875,0.114392
"(0.3, 0.4]",111.0,48.0,38.956,0.059974,0.134100
"(0.4, 0.5]",151.0,69.0,67.603,0.753942,0.802662
"(0.5, 0.6]",183.0,93.0,100.512,0.296886,0.489605
"(0.6, 0.7]",144.0,90.0,93.494,0.596220,0.768761
"(0.7, 0.8]",122.0,80.0,91.069,0.031570,0.266043
"(0.8, 0.9]",76.0,51.0,64.672,0.000127,0.093474
"(0.9, 1.0]",41.0,34.0,38.767,0.011600,0.502123


ls = 2


...

,n,obs,exp,poibin_pval,pois_pval
"(-0.001, 0.1]",0.0,0.0,0.000,-4.440892e-16,0.000000
"(0.1, 0.2]",4.0,1.0,0.724,3.043306e-01,0.328374
"(0.2, 0.3]",29.0,9.0,7.447,3.772252e-01,0.435146
"(0.3, 0.4]",116.0,33.0,41.223,1.297232e-01,0.223691
"(0.4, 0.5]",266.0,109.0,120.787,1.632498e-01,0.303844
"(0.5, 0.6]",262.0,143.0,144.330,9.161603e-01,0.955953
"(0.6, 0.7]",229.0,151.0,147.413,5.752019e-01,0.727263
"(0.7, 0.8]",76.0,58.0,56.213,5.585960e-01,0.745080
"(0.8, 0.9]",17.0,13.0,14.191,6.103194e-01,0.888599
"(0.9, 1.0]",1.0,1.0,0.934,-8.881784e-16,0.480076


ls = 4.333 (new)


,n,obs,exp,poibin_pval,pois_pval
"(-0.001, 0.1]",0.0,0.0,0.000,-4.440892e-16,0.000000
"(0.1, 0.2]",2.0,1.0,0.369,6.790593e-02,0.107113
"(0.2, 0.3]",30.0,8.0,7.849,7.635081e-01,0.772882
"(0.3, 0.4]",106.0,31.0,38.066,1.801280e-01,0.284947
"(0.4, 0.5]",270.0,109.0,122.878,1.007276e-01,0.224569
"(0.5, 0.6]",282.0,154.0,155.493,9.036175e-01,0.947111
"(0.6, 0.7]",228.0,151.0,146.996,5.351896e-01,0.701515
"(0.7, 0.8]",64.0,50.0,47.168,3.445218e-01,0.614520
"(0.8, 0.9]",17.0,13.0,14.008,6.998869e-01,0.927121
"(0.9, 1.0]",1.0,1.0,0.927,-8.881784e-16,0.474532


ls = 4.667 (new)


,n,obs,exp,poibin_pval,pois_pval
"(-0.001, 0.1]",0.0,0.0,0.000,-4.440892e-16,0.000000
"(0.1, 0.2]",1.0,0.0,0.187,3.741849e-01,0.341266
"(0.2, 0.3]",24.0,9.0,6.234,1.367197e-01,0.201903
"(0.3, 0.4]",106.0,28.0,38.146,4.668796e-02,0.107974
"(0.4, 0.5]",271.0,110.0,123.861,1.019628e-01,0.227107
"(0.5, 0.6]",293.0,160.0,161.308,9.227562e-01,0.959711
"(0.6, 0.7]",230.0,151.0,147.846,6.180509e-01,0.754327
"(0.7, 0.8]",62.0,50.0,45.717,1.589942e-01,0.471637
"(0.8, 0.9]",12.0,9.0,9.859,7.289598e-01,0.951337
"(0.9, 1.0]",1.0,1.0,0.919,-8.881784e-16,0.468750


### Varying time scale

In [29]:
for label, match_pred in match_pred_objs.items():
    if not label.startswith('ts'):
        continue
    print(label)
    cur_loc = match_pred.df.iter > 0
    display(match_pred.match_pred_df_bias(loc=cur_loc,
                                          assume_side_known=True))

ts = 0.5


,n,obs,exp,poibin_pval,pois_pval
"(-0.001, 0.1]",0.0,0.0,0.000,-4.440892e-16,0.000000
"(0.1, 0.2]",6.0,3.0,1.027,1.885077e-02,0.041361
"(0.2, 0.3]",40.0,9.0,10.223,8.150828e-01,0.860826
"(0.3, 0.4]",115.0,31.0,40.992,6.025768e-02,0.128800
"(0.4, 0.5]",253.0,116.0,115.160,8.640514e-01,0.888545
"(0.5, 0.6]",281.0,152.0,155.283,7.366987e-01,0.833215
"(0.6, 0.7]",198.0,124.0,128.008,5.978537e-01,0.766726
"(0.7, 0.8]",82.0,65.0,61.050,2.559868e-01,0.559241
"(0.8, 0.9]",24.0,17.0,20.128,1.572899e-01,0.574821
"(0.9, 1.0]",1.0,1.0,0.946,-8.881784e-16,0.488533


ts = 0.75


,n,obs,exp,poibin_pval,pois_pval
"(-0.001, 0.1]",0.0,0.0,0.000,-4.440892e-16,0.000000
"(0.1, 0.2]",10.0,3.0,1.746,1.600900e-01,0.200528
"(0.2, 0.3]",43.0,11.0,11.196,8.939276e-01,0.888146
"(0.3, 0.4]",121.0,35.0,43.098,1.452255e-01,0.242862
"(0.4, 0.5]",246.0,112.0,111.387,8.849103e-01,0.903609
"(0.5, 0.6]",266.0,139.0,147.221,3.399973e-01,0.529925
"(0.6, 0.7]",198.0,129.0,128.437,8.797072e-01,0.913692
"(0.7, 0.8]",90.0,70.0,67.172,4.230375e-01,0.672335
"(0.8, 0.9]",23.0,16.0,19.309,1.255237e-01,0.537442
"(0.9, 1.0]",3.0,3.0,2.766,-1.776357e-15,0.601173


ts = 1


...

,n,obs,exp,poibin_pval,pois_pval
"(-0.001, 0.1]",1.0,0.0,0.095,1.893343e-01,0.180649
"(0.1, 0.2]",17.0,6.0,2.957,3.647059e-02,0.062749
"(0.2, 0.3]",56.0,13.0,14.657,7.411086e-01,0.793441
"(0.3, 0.4]",129.0,44.0,46.217,7.582176e-01,0.818537
"(0.4, 0.5]",237.0,103.0,106.588,6.877412e-01,0.776176
"(0.5, 0.6]",195.0,104.0,107.200,6.957002e-01,0.805906
"(0.6, 0.7]",215.0,133.0,138.634,4.618069e-01,0.671186
"(0.7, 0.8]",110.0,83.0,81.354,6.502190e-01,0.798444
"(0.8, 0.9]",35.0,27.0,29.465,3.556164e-01,0.737717
"(0.9, 1.0]",5.0,5.0,4.608,-2.664535e-15,0.631322


ts = 3 (new)


,n,obs,exp,poibin_pval,pois_pval
"(-0.001, 0.1]",1.0,0.0,0.094,1.876681e-01,0.179132
"(0.1, 0.2]",17.0,6.0,2.938,3.529107e-02,0.060997
"(0.2, 0.3]",54.0,12.0,13.971,6.620092e-01,0.722607
"(0.3, 0.4]",133.0,46.0,47.539,8.571565e-01,0.898954
"(0.4, 0.5]",234.0,102.0,105.180,7.257697e-01,0.805598
"(0.5, 0.6]",195.0,103.0,107.075,6.052129e-01,0.740458
"(0.6, 0.7]",218.0,134.0,140.862,3.660895e-01,0.598914
"(0.7, 0.8]",106.0,81.0,78.414,4.997326e-01,0.715115
"(0.8, 0.9]",37.0,29.0,31.101,4.557526e-01,0.795334
"(0.9, 1.0]",5.0,5.0,4.610,-2.664535e-15,0.632086


ts = 3.25 (new)


,n,obs,exp,poibin_pval,pois_pval
"(-0.001, 0.1]",1.0,0.0,0.093,1.862600e-01,0.177850
"(0.1, 0.2]",17.0,6.0,2.923,3.432763e-02,0.059560
"(0.2, 0.3]",54.0,12.0,13.899,6.783820e-01,0.737021
"(0.3, 0.4]",133.0,46.0,47.446,8.705281e-01,0.909712
"(0.4, 0.5]",230.0,100.0,103.077,7.337249e-01,0.811553
"(0.5, 0.6]",200.0,106.0,109.660,6.516718e-01,0.773933
"(0.6, 0.7]",219.0,134.0,141.820,2.998777e-01,0.544556
"(0.7, 0.8]",104.0,80.0,77.047,4.430940e-01,0.682266
"(0.8, 0.9]",37.0,29.0,31.128,4.483408e-01,0.791742
"(0.9, 1.0]",5.0,5.0,4.612,-2.664535e-15,0.632683
